In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import ssl
import os
from bs4 import BeautifulSoup
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
from urllib.parse import urlparse

#DEFINE FUNCTIONS

In [ ]:
# Read Url list from csv
def load_urls(file_path):
    urls = []
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row:  # Kiểm tra nếu dòng không rỗng
                urls.append(row[0])
    return urls

In [ ]:
# Create SSLContext to avoid DH_KEY_TOO_SMALL error
def create_ssl_context():
    context = ssl.create_default_context()
    context.set_ciphers('ALL')
    return context

In [ ]:
# Reload page và save HTML
def download_page(url):
    try:
        context = create_ssl_context()
        response = requests.get(url, verify=False, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error when dowwloading page {url}: {e}")
        return None

In [ ]:
# Add scheme into URL
def add_scheme_to_url(url):
    parsed_url = urlparse(url)
    if not parsed_url.scheme:
        return 'http://' + url
    return url

In [ ]:
# Check url is valid or not
def is_valid_url(url):
    parsed_url = urlparse(url)
    return bool(parsed_url.scheme and parsed_url.netloc)


def sanitize_url(url):
    sanitized_url = url.rstrip('.')
    if sanitized_url.count('.') < 2:
        sanitized_url = sanitized_url + ".gob.pe"
    return sanitized_url

In [ ]:
# Take the text from .html
def extract_text_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Take <body>
    body_text = soup.body.get_text(separator=' ', strip=True) if soup.body else ""

    # Take <title>
    if not body_text:
        title = soup.title.get_text(separator=' ', strip=True) if soup.title else ""
        body_text = title

    # Take main content(ví dụ: <article>, <section>, <p>, <h1>, <h2>, <h3>)
    if not body_text:
        paragraphs = soup.find_all(['article', 'section', 'p', 'h1', 'h2', 'h3'])
        if paragraphs:
            body_text = ' '.join([p.get_text(separator=' ', strip=True) for p in paragraphs])

    # Take <header>, <footer> nếu cần
    if not body_text:
        header_footer = soup.find_all(['header', 'footer'])
        if header_footer:
            body_text = ' '.join([hf.get_text(separator=' ', strip=True) for hf in header_footer])

    # Take <meta name="description">
    if not body_text:
        meta_desc = soup.find('meta', {'name': 'description'})
        if meta_desc and meta_desc.get('content'):
            body_text = meta_desc['content']

    # Take <main>
    if not body_text:
        main_content = soup.find('main')
        if main_content:
            body_text = main_content.get_text(separator=' ', strip=True)

    # Take others
    if not body_text:
        body_text = soup.get_text(separator=' ', strip=True)

    return clean_text(body_text)

In [ ]:
# Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [ ]:
# Save text into folder
def save_text_to_file(url, text, folder):
    # Tách phần domain và phần đường dẫn trong URL
    domain = url.split('//')[1].split('/')[0]
    path = url.split(domain)[1].replace('/', '_')  # Thay dấu '/' bằng dấu '_' để tránh vấn đề tên tệp

    # Kết hợp domain và path thành tên tệp duy nhất
    filename = f"{domain}{path}"

    # Giới hạn độ dài tên tệp để tránh vượt quá giới hạn của hệ thống tệp (255 ký tự)
    max_filename_length = 200
    if len(filename) > max_filename_length:
        filename = filename[:max_filename_length]  # Cắt tên tệp nếu quá dài

    # Lưu văn bản vào file .txt
    filepath = os.path.join(folder, f"{filename}.txt")
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(text)

    return filepath


In [ ]:
# sace URL does not load into fail.csv
def save_failed_urls():
    if failed_urls:
        with open('fail.csv', 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Failed URLs'])  # Tiêu đề cột
            for url in failed_urls:
                writer.writerow([url])

#Download Pages and Save Content

In [ ]:
# Create folder to save output, including safe and defaced folders
output_folder = 'output'
safe_folder = os.path.join(output_folder, 'safe')
deface_folder = os.path.join(output_folder, 'deface')

# Create subfolders if they do not exist
if not os.path.exists(safe_folder):
    os.makedirs(safe_folder)
if not os.path.exists(deface_folder):
    os.makedirs(deface_folder)

# Save URLs that failed to load into fail.csv
failed_urls = []

# Read URLs from CSV file
safe_urls = load_urls('/content/safe_urls.csv')
deface_urls = load_urls('/content/deface_urls.csv')

# Reload pages and save output
safe_documents = []
deface_documents = []

# Load pages and save to safe and deface folders
for url in safe_urls:
    url = sanitize_url(url)
    url = add_scheme_to_url(url)
    if not is_valid_url(url):
        failed_urls.append(url)
        print(f"Invalid URL: {url}")
        continue
    print(f"Downloading page {url}...")
    html_content = download_page(url)
    if html_content:
        body_text = extract_text_from_html(html_content)

        if body_text:
            filename = save_text_to_file(url, body_text, safe_folder)
            safe_documents.append(filename)  # Save .txt
            print(f"Text saved to {filename}")
        else:
            failed_urls.append(url)
            print(f"No valid text from {url}")
    else:
        failed_urls.append(url)
        print(f"Failed to download page from {url}")

for url in deface_urls:
    url = sanitize_url(url)
    url = add_scheme_to_url(url)
    if not is_valid_url(url):
        failed_urls.append(url)
        print(f"Invalid URL: {url}")
        continue
    print(f"Downloading page {url}...")
    html_content = download_page(url)
    if html_content:
        body_text = extract_text_from_html(html_content)

        if body_text:
            filename = save_text_to_file(url, body_text, deface_folder)
            deface_documents.append(filename)
            print(f"Text saved to {filename}")
        else:
            failed_urls.append(url)
            print(f"No valid text from {url}")
    else:
        failed_urls.append(url)
        print(f"Failed to download page from {url}")

# Save failed URLs to fail.csv
save_failed_urls()


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'moj.gov.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'moj.gov.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Text saved to output/safe/moj.gov.vn_.txt


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ncov.moh.gov.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error when dowwloading page https://ncov.moh.gov.vn/: HTTPSConnectionPool(host='covid19.gov.vn', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7e1484878250>: Failed to resolve 'covid19.gov.vn' ([Errno -2] Name or service not known)"))
Failed to download page from https://ncov.moh.gov.vn/


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'chinhphu.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Text saved to output/deface/empalmevillaconstitucion.gob.ar.txt


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'durande.mg.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error when dowwloading page http://durande.mg.gov.br/mrz.php: 404 Client Error: Not Found for url: https://durande.mg.gov.br/mrz.php
Failed to download page from http://durande.mg.gov.br/mrz.php
Error when dowwloading page http://www.kukarkab.go.id/asu.txt: HTTPConnectionPool(host='www.kukarkab.go.id', port=80): Max retries exceeded with url: /asu.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7e148406a740>, 'Connection to www.kukarkab.go.id timed out. (connect timeout=10)'))
Failed to download page from http://www.kukarkab.go.id/asu.txt


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dgaie.gov.bf'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error when dowwloading page http://dgaie.gov.bf/kurd.html: HTTPSConnectionPool(host='dgaie.gov.bf', port=443): Read timed out. (read timeout=10)
Failed to download page from http://dgaie.gov.bf/kurd.html


#Text Preprocessing & TF-IDF Calculation

In [ ]:
import os
import csv
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Path to the folders containing the downloaded files
output_folder = '/content/drive/MyDrive/output'
safe_folder = os.path.join(output_folder, 'safe')
deface_folder = os.path.join(output_folder, 'deface')

# Function to get all .txt files in a folder
def get_text_files_from_folder(folder):
    text_files = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.txt'):  # Check if the file is a .txt file
                text_files.append(os.path.join(root, file))
    return text_files

# Preprocessing the text: removing punctuation and converting to lowercase
def preprocess_text(text):
    text = text.lower()  # Convert all to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove all punctuation
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text

# Read the text files from the "safe" and "deface" folders
safe_documents = get_text_files_from_folder(safe_folder)
deface_documents = get_text_files_from_folder(deface_folder)

# Preprocess the text and assign labels
all_documents = []
labels = []

print("Processing the documents...")

# Read and preprocess documents from the "safe" folder
for doc in safe_documents:
    with open(doc, 'r', encoding='utf-8') as f:
        text = f.read()  # Read the content of the .txt file
        processed_text = preprocess_text(text)  # Preprocess the text
        all_documents.append(processed_text)
        labels.append(0)  # Label for "safe" is 0

# Read and preprocess documents from the "deface" folder
for doc in deface_documents:
    with open(doc, 'r', encoding='utf-8') as f:
        text = f.read()  # Read the content of the .txt file
        processed_text = preprocess_text(text)  # Preprocess the text
        all_documents.append(processed_text)
        labels.append(1)  # Label for "deface" is 1

# Calculate TF-IDF for all the documents
print("Calculating TF-IDF...")
vectorizer = TfidfVectorizer(max_features=300, ngram_range=(1, 2), stop_words=None)  # Using unigrams and bigrams
tfidf_matrix = vectorizer.fit_transform(all_documents)

# Convert the sparse (CSR) matrix to a pandas DataFrame
X_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

Processing the documents...
Calculating TF-IDF...


#TRAIN MODELS

In [ ]:
!pip install lazypredict

In [ ]:
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier  # Using LazyPredict
from sklearn.metrics import accuracy_score
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, labels, test_size=0.2, random_state=15)

# Use LazyPredict to try different machine learning models
print("Using LazyPredict to train models...")
clf = LazyClassifier()
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Print the results from LazyPredict
print(models)  # Display the results of models with their accuracies

# Save the predictions to a CSV file
predictions.to_csv('lazy_predictions.csv', index=False)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Using LazyPredict to train models...


 97%|█████████▋| 30/31 [00:28<00:00,  4.49it/s]

[LightGBM] [Info] Number of positive: 250, number of negative: 862
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003699 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 18855
[LightGBM] [Info] Number of data points in the train set: 1112, number of used features: 283
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.224820 -> initscore=-1.237794
[LightGBM] [Info] Start training from score -1.237794
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


100%|██████████| 31/31 [00:29<00:00,  1.06it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
RandomForestClassifier             0.94               0.89     0.89      0.94   
PassiveAggressiveClassifier        0.94               0.89     0.89      0.94   
ExtraTreesClassifier               0.94               0.89     0.89      0.94   
XGBClassifier                      0.94               0.88     0.88      0.94   
LogisticRegression                 0.94               0.88     0.88      0.93   
LinearSVC                          0.92               0.88     0.88      0.92   
LinearDiscriminantAnalysis         0.94               0.87     0.87      0.93   
RidgeClassifier                    0.94       

#RANDOM FOREST MODEL

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Create the RandomForestClassifier model
model = RandomForestClassifier()

# Train the model on the training set
print("Training the RandomForestClassifier model...")
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the RandomForestClassifier model: {accuracy * 100:.2f}%")


Training the RandomForestClassifier model...
Accuracy of the RandomForestClassifier model: 93.53%


#SAVE MODELS

In [ ]:
import joblib  # For saving the model and vectorizer

In [ ]:

# Save the model and vectorizer
joblib.dump(model, "/content/drive/MyDrive/model/random_forest_model.pkl")
joblib.dump(vectorizer, "/content/drive/MyDrive/model/tfidf_vectorizer.pkl")

['/content/drive/MyDrive/model/tfidf_vectorizer.pkl']

#PREDICT

In [ ]:
# Function to predict the class of a new URL
def predict_url(url, model, vectorizer):
    # Download the webpage content
    html_content = download_page(url)

    if html_content:
        # Extract text from HTML
        body_text = extract_text_from_html(html_content)

        # Preprocess the text
        processed_text = preprocess_text(body_text)

        # Convert the text to a TF-IDF vector
        tfidf_vector = vectorizer.transform([processed_text])

        # Predict the label (0: safe, 1: deface)
        prediction = model.predict(tfidf_vector)

        return prediction[0]  # Return the predicted label (0 or 1)
    else:
        return None  # If the page cannot be loaded, return None

In [ ]:
# Example URL prediction
test_url = "https://vnexpress.net/"  # Replace this URL with the one you want to check
prediction = predict_url(test_url, model, vectorizer)

if prediction is not None:
    print(f"URL: {test_url}")
    if prediction == 0:
        print("Predicted class: Safe")
    else:
        print("Predicted class: Deface")
else:
    print(f"Could not download the page from {test_url}")

URL: https://vnexpress.net/
Predicted class: Safe


In [ ]:
# Example URL prediction
test_url = "https://primarias.voto/index.html"  # Replace this URL with the one you want to check
prediction = predict_url(test_url, model, vectorizer)

if prediction is not None:
    print(f"URL: {test_url}")
    if prediction == 0:
        print("Predicted class: Safe")
    else:
        print("Predicted class: Deface")
else:
    print(f"Could not download the page from {test_url}")


URL: https://primarias.voto/index.html
Predicted class: Deface


#WEB


In [ ]:
!pip install pyngrok


In [ ]:
from flask import Flask, request, render_template, jsonify
from pyngrok import ngrok
import joblib
import requests
from bs4 import BeautifulSoup
import re
import os
# Ensure the URL has a proper scheme
from urllib.parse import urlparse, unquote

# Load the model and vectorizer
model = joblib.load("/content/drive/MyDrive/model/random_forest_model.pkl")  # Ensure the file is uploaded correctly in Colab
vectorizer = joblib.load("/content/drive/MyDrive/model/tfidf_vectorizer.pkl")

# Flask app
app = Flask(__name__, template_folder='/content/drive/MyDrive')  # Provide the correct template folder path

def download_page(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error when downloading page {url}: {e}")
        return None

# Preprocessing function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove all punctuation
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text

# Extract text from HTML content
def extract_text_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract <body>
    body_text = soup.body.get_text(separator=' ', strip=True) if soup.body else ""

    # Extract <title>
    if not body_text:
        title = soup.title.get_text(separator=' ', strip=True) if soup.title else ""
        body_text = title

    # Extract main content (e.g., <article>, <section>, <p>, <h1>, <h2>, <h3>)
    if not body_text:
        paragraphs = soup.find_all(['article', 'section', 'p', 'h1', 'h2', 'h3'])
        if paragraphs:
            body_text = ' '.join([p.get_text(separator=' ', strip=True) for p in paragraphs])

    # Extract <header>, <footer> if necessary
    if not body_text:
        header_footer = soup.find_all(['header', 'footer'])
        if header_footer:
            body_text = ' '.join([hf.get_text(separator=' ', strip=True) for hf in header_footer])

    # Extract <meta name="description">
    if not body_text:
        meta_desc = soup.find('meta', {'name': 'description'})
        if meta_desc and meta_desc.get('content'):
            body_text = meta_desc['content']

    # Extract <main>
    if not body_text:
        main_content = soup.find('main')
        if main_content:
            body_text = main_content.get_text(separator=' ', strip=True)

    # Extract others if nothing is found
    if not body_text:
        body_text = soup.get_text(separator=' ', strip=True)

    return clean_text(body_text)

def preprocess_text(text):
    text = text.lower()  # Convert all to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove all punctuation
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text



def ensure_https(url):
    # Remove spaces and decode encoded spaces
    url = unquote(url).strip()  # Decode '%20' into actual spaces and remove unnecessary spaces
    parsed_url = urlparse(url)

    if not parsed_url.scheme:
        url = "https://" + url
    return url



@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        url = request.form.get("url", "")
        if not url:
            return render_template("index.html", result="URL is required.", result_class="error")

        # Ensure URL has the correct scheme
        url = ensure_https(url)

        html_content = download_page(url)
        if html_content:
            body_text = extract_text_from_html(html_content)
            processed_text = preprocess_text(body_text)

            tfidf_vector = vectorizer.transform([processed_text])
            prediction = model.predict(tfidf_vector)[0]

            if prediction == 0:
                result = "Safe (No Defacement Detected)"
                result_class = "safe"
            else:
                result = "Deface Detected!"
                result_class = "deface"
        else:
            result = f"Could not download the page from {url}."
            result_class = "error"

        return render_template("index.html", result=result, result_class=result_class)
    return render_template("index.html", result=None)


if __name__ == "__main__":
    ngrok.set_auth_token("2q1Br34fuBE00tx7ALzcygJbJOG_gurkXnznNgBjaj1e5mAU")
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run()


Public URL: NgrokTunnel: "https://9006-34-16-186-74.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 13:59:10] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 13:59:10] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 13:59:15] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 13:59:28] "POST / HTTP/1.1" 200 -
